In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"insurance.csv")
df

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86
...,...,...,...,...,...,...,...
1333,50,male,31.0,3,no,northwest,10600.55
1334,18,female,31.9,0,no,northeast,2205.98
1335,18,female,36.9,0,no,southeast,1629.83
1336,21,female,25.8,0,no,southwest,2007.95


In [2]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each categorical column
df['sex'] = label_encoder.fit_transform(df['sex'])



In [3]:
print("Mapping for 'sex':")
for i, category in enumerate(label_encoder.classes_):
    print(f"{category} -> {i}")

Mapping for 'sex':
female -> 0
male -> 1


In [4]:
df['smoker'] = label_encoder.fit_transform(df['smoker'])

In [5]:
print("\nMapping for 'smoker':")
for i, category in enumerate(label_encoder.classes_):
    print(f"{category} -> {i}")


Mapping for 'smoker':
no -> 0
yes -> 1


In [6]:
df['region'] = label_encoder.fit_transform(df['region'])

In [7]:
print("\nMapping for 'region':")
for i, category in enumerate(label_encoder.classes_):
    print(f"{category} -> {i}")


Mapping for 'region':
northeast -> 0
northwest -> 1
southeast -> 2
southwest -> 3


In [8]:
df

,age,sex,bmi,children,smoker,region,expenses
0,19,0,27.9,0,1,3,16884.92
1,18,1,33.8,1,0,2,1725.55
2,28,1,33.0,3,0,2,4449.46
3,33,1,22.7,0,0,1,21984.47
4,32,1,28.9,0,0,1,3866.86
...,...,...,...,...,...,...,...
1333,50,1,31.0,3,0,1,10600.55
1334,18,0,31.9,0,0,0,2205.98
1335,18,0,36.9,0,0,2,1629.83
1336,21,0,25.8,0,0,3,2007.95


In [9]:
from sklearn.model_selection import train_test_split , GridSearchCV

X = df.drop(['expenses'], axis=1)
y = df['expenses']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import make_scorer


## checking the r2 score for diff regressors before doing hyper parameter tunning

In [11]:


regression_models = {
    "Linear Regression": LinearRegression(),
    "K-Nearest Neighbors": KNeighborsRegressor(),
    "Decision Tree Regressor": DecisionTreeRegressor(max_depth=7),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBoost Regressor": XGBRegressor(),
    "CatBoost Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

for model_name, model in regression_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    

    print(f"Model: {model_name}")
    print(f"R^2 Score: {r2:.5f}")
    print(f"Mean Squared Error: {mse:.5f}")
    print(f"Mean Absolute Error: {mae:.5f}")
    
    print('***********************************')


Model: Linear Regression
R^2 Score: 0.76946
Mean Squared Error: 33802383.55494
Mean Absolute Error: 4154.70039
***********************************
Model: K-Nearest Neighbors
R^2 Score: 0.12143
Mean Squared Error: 128819314.42014
Mean Absolute Error: 7893.72658
***********************************
Model: Decision Tree Regressor
R^2 Score: 0.80524
Mean Squared Error: 28556820.90761
Mean Absolute Error: 2866.02214
***********************************
Model: Random Forest Regressor
R^2 Score: 0.85003
Mean Squared Error: 21988838.51332
Mean Absolute Error: 2572.08512
***********************************
Model: XGBoost Regressor
R^2 Score: 0.82005
Mean Squared Error: 26384383.00783
Mean Absolute Error: 2872.00475
***********************************
Model: CatBoost Regressor
R^2 Score: 0.85885
Mean Squared Error: 20696680.90840
Mean Absolute Error: 2539.48901
***********************************
Model: AdaBoost Regressor
R^2 Score: 0.82547
Mean Squared Error: 25590650.50200
Mean Absolute Error: 4

# # Hyper parameter tunning

In [12]:
model = LinearRegression()

parameters = {'fit_intercept': [True, False]}
scorer = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = GridSearchCV(model, parameters, scoring=scorer, cv=5)
grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_
print("Best Parameters:", best_params)
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Model -  R^2: {r2:.5f}, MSE: {mse:.5f}, MAE: {mae:.5f}")



Best Parameters: {'fit_intercept': True}
Best Model -  R^2: 0.76946, MSE: 33802383.55494, MAE: 4034.93920


In [13]:

param_grid = {
    'iterations': [100, 200, 300],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1]
}
catboost_regressor = CatBoostRegressor()

grid_search = GridSearchCV(catboost_regressor, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Model -  R^2: {r2:.5f}, MSE: {mse:.5f}, MAE: {mae:.5f}")


0:	learn: 11937.3150519	total: 950us	remaining: 94.1ms
1:	learn: 11856.7986801	total: 1.63ms	remaining: 80ms
2:	learn: 11769.8301260	total: 2.31ms	remaining: 74.5ms
3:	learn: 11701.6519614	total: 2.83ms	remaining: 67.8ms
4:	learn: 11626.9871421	total: 3.41ms	remaining: 64.9ms
5:	learn: 11543.4136995	total: 3.91ms	remaining: 61.3ms
6:	learn: 11457.5496707	total: 4.39ms	remaining: 58.4ms
7:	learn: 11378.0360672	total: 4.89ms	remaining: 56.2ms
8:	learn: 11300.5068858	total: 5.91ms	remaining: 59.8ms
9:	learn: 11218.8504654	total: 8ms	remaining: 72ms
10:	learn: 11155.6734734	total: 8.55ms	remaining: 69.2ms
11:	learn: 11079.9526452	total: 9.12ms	remaining: 66.9ms
12:	learn: 11003.2760417	total: 10ms	remaining: 67ms
13:	learn: 10924.7701730	total: 10.5ms	remaining: 64.5ms
14:	learn: 10849.6308064	total: 11ms	remaining: 62.4ms
15:	learn: 10781.1718529	total: 11.5ms	remaining: 60.4ms
16:	learn: 10703.9808533	total: 12.1ms	remaining: 58.9ms
17:	learn: 10632.9852168	total: 14.7ms	remaining: 66.8m

92:	learn: 6915.5127928	total: 59.3ms	remaining: 4.46ms
93:	learn: 6884.6959432	total: 59.9ms	remaining: 3.83ms
94:	learn: 6854.4792188	total: 60.5ms	remaining: 3.19ms
95:	learn: 6830.5653018	total: 61ms	remaining: 2.54ms
96:	learn: 6804.5781156	total: 61.5ms	remaining: 1.9ms
97:	learn: 6777.8679309	total: 62.2ms	remaining: 1.27ms
98:	learn: 6749.5861025	total: 62.7ms	remaining: 633us
99:	learn: 6724.5243573	total: 63.4ms	remaining: 0us
0:	learn: 12322.7238939	total: 1.21ms	remaining: 120ms
1:	learn: 12237.0809157	total: 1.81ms	remaining: 89ms
2:	learn: 12145.4840630	total: 3.52ms	remaining: 114ms
3:	learn: 12071.5732595	total: 4.14ms	remaining: 99.5ms
4:	learn: 11986.8589840	total: 4.89ms	remaining: 92.8ms
5:	learn: 11897.0371179	total: 5.38ms	remaining: 84.3ms
6:	learn: 11804.5942069	total: 5.83ms	remaining: 77.5ms
7:	learn: 11718.9909159	total: 6.29ms	remaining: 72.3ms
8:	learn: 11636.8139315	total: 6.76ms	remaining: 68.4ms
9:	learn: 11547.3455306	total: 7.25ms	remaining: 65.3ms
10:

39:	learn: 9369.9555283	total: 25.7ms	remaining: 38.6ms
40:	learn: 9315.3671559	total: 27.7ms	remaining: 39.9ms
41:	learn: 9261.7346969	total: 28.3ms	remaining: 39.1ms
42:	learn: 9209.4222054	total: 28.8ms	remaining: 38.2ms
43:	learn: 9156.0744334	total: 29.6ms	remaining: 37.6ms
44:	learn: 9097.2846633	total: 30.1ms	remaining: 36.8ms
45:	learn: 9046.4516307	total: 30.6ms	remaining: 35.9ms
46:	learn: 8991.6950444	total: 31.3ms	remaining: 35.3ms
47:	learn: 8942.8444823	total: 33ms	remaining: 35.7ms
48:	learn: 8885.5656707	total: 33.6ms	remaining: 34.9ms
49:	learn: 8830.4099058	total: 34ms	remaining: 34ms
50:	learn: 8773.6105265	total: 34.5ms	remaining: 33.1ms
51:	learn: 8717.0512428	total: 34.9ms	remaining: 32.2ms
52:	learn: 8666.5142209	total: 35.2ms	remaining: 31.2ms
53:	learn: 8616.3756765	total: 35.7ms	remaining: 30.4ms
54:	learn: 8560.5448764	total: 36.3ms	remaining: 29.7ms
55:	learn: 8507.8449946	total: 36.9ms	remaining: 29ms
56:	learn: 8462.4399610	total: 37.3ms	remaining: 28.1ms


92:	learn: 6900.9002284	total: 55ms	remaining: 4.14ms
93:	learn: 6868.7339602	total: 57.2ms	remaining: 3.65ms
94:	learn: 6836.2650065	total: 58.1ms	remaining: 3.06ms
95:	learn: 6802.2385574	total: 58.7ms	remaining: 2.44ms
96:	learn: 6771.5587074	total: 59.1ms	remaining: 1.83ms
97:	learn: 6739.8056989	total: 59.6ms	remaining: 1.22ms
98:	learn: 6707.4380993	total: 60.1ms	remaining: 606us
99:	learn: 6678.1221089	total: 60.5ms	remaining: 0us
0:	learn: 11618.9920510	total: 962us	remaining: 95.3ms
1:	learn: 11237.1533607	total: 1.75ms	remaining: 86ms
2:	learn: 10835.3154629	total: 2.31ms	remaining: 74.6ms
3:	learn: 10541.6294033	total: 2.68ms	remaining: 64.3ms
4:	learn: 10231.8435220	total: 3.38ms	remaining: 64.2ms
5:	learn: 9889.5816918	total: 4.67ms	remaining: 73.1ms
6:	learn: 9549.9752615	total: 5.43ms	remaining: 72.1ms
7:	learn: 9247.5528758	total: 5.99ms	remaining: 68.9ms
8:	learn: 8951.8350685	total: 6.55ms	remaining: 66.3ms
9:	learn: 8665.8595029	total: 7.13ms	remaining: 64.2ms
10:	le

46:	learn: 4918.2437047	total: 25.3ms	remaining: 28.5ms
47:	learn: 4891.3317963	total: 25.8ms	remaining: 27.9ms
48:	learn: 4871.1668748	total: 26.3ms	remaining: 27.4ms
49:	learn: 4850.8086175	total: 26.8ms	remaining: 26.8ms
50:	learn: 4829.8412724	total: 27.3ms	remaining: 26.2ms
51:	learn: 4818.1186471	total: 27.6ms	remaining: 25.5ms
52:	learn: 4799.9132503	total: 28.1ms	remaining: 24.9ms
53:	learn: 4784.7220654	total: 28.6ms	remaining: 24.4ms
54:	learn: 4771.7569176	total: 29.1ms	remaining: 23.8ms
55:	learn: 4759.4249761	total: 29.7ms	remaining: 23.3ms
56:	learn: 4747.4587137	total: 30.1ms	remaining: 22.7ms
57:	learn: 4734.2170142	total: 30.7ms	remaining: 22.2ms
58:	learn: 4726.2311261	total: 31.2ms	remaining: 21.7ms
59:	learn: 4711.1888525	total: 31.8ms	remaining: 21.2ms
60:	learn: 4702.4189716	total: 32.3ms	remaining: 20.6ms
61:	learn: 4689.5157243	total: 32.8ms	remaining: 20.1ms
62:	learn: 4678.2242158	total: 33.3ms	remaining: 19.6ms
63:	learn: 4663.8324289	total: 33.9ms	remaining:

...

45:	learn: 3891.0367921	total: 86.9ms	remaining: 480ms
46:	learn: 3873.1396156	total: 89.2ms	remaining: 480ms
47:	learn: 3847.0611640	total: 91.5ms	remaining: 480ms
48:	learn: 3822.1837219	total: 93.6ms	remaining: 480ms
49:	learn: 3817.3967711	total: 94.2ms	remaining: 471ms
50:	learn: 3796.5910331	total: 96.4ms	remaining: 471ms
51:	learn: 3783.0163759	total: 98.7ms	remaining: 471ms
52:	learn: 3765.6365721	total: 101ms	remaining: 470ms
53:	learn: 3749.2401054	total: 103ms	remaining: 469ms
54:	learn: 3730.3104161	total: 105ms	remaining: 468ms
55:	learn: 3722.5741240	total: 107ms	remaining: 468ms
56:	learn: 3701.0953249	total: 110ms	remaining: 467ms
57:	learn: 3683.3389581	total: 112ms	remaining: 467ms
58:	learn: 3671.2763973	total: 114ms	remaining: 466ms
59:	learn: 3657.8031342	total: 116ms	remaining: 465ms
60:	learn: 3650.7771514	total: 118ms	remaining: 464ms
61:	learn: 3635.8868869	total: 120ms	remaining: 462ms
62:	learn: 3620.1538449	total: 123ms	remaining: 461ms
63:	learn: 3603.69046

195:	learn: 2620.7096257	total: 440ms	remaining: 234ms
196:	learn: 2613.7666156	total: 443ms	remaining: 231ms
197:	learn: 2603.6208226	total: 445ms	remaining: 229ms
198:	learn: 2592.4112804	total: 447ms	remaining: 227ms
199:	learn: 2587.3087850	total: 449ms	remaining: 225ms
200:	learn: 2574.0780572	total: 451ms	remaining: 222ms
201:	learn: 2568.1984523	total: 454ms	remaining: 220ms
202:	learn: 2561.2159121	total: 456ms	remaining: 218ms
203:	learn: 2552.0024368	total: 459ms	remaining: 216ms
204:	learn: 2540.0555445	total: 461ms	remaining: 214ms
205:	learn: 2538.7349458	total: 463ms	remaining: 211ms
206:	learn: 2531.0448526	total: 465ms	remaining: 209ms
207:	learn: 2523.7628837	total: 467ms	remaining: 207ms
208:	learn: 2522.9148067	total: 470ms	remaining: 204ms
209:	learn: 2511.8592589	total: 472ms	remaining: 202ms
210:	learn: 2505.5989754	total: 474ms	remaining: 200ms
211:	learn: 2504.7727403	total: 476ms	remaining: 198ms
212:	learn: 2501.1070180	total: 478ms	remaining: 195ms
213:	learn

44:	learn: 3885.4856821	total: 82.4ms	remaining: 467ms
45:	learn: 3871.8655965	total: 84.7ms	remaining: 468ms
46:	learn: 3852.6668898	total: 86.9ms	remaining: 468ms
47:	learn: 3827.0230100	total: 89ms	remaining: 467ms
48:	learn: 3815.0386429	total: 90.4ms	remaining: 463ms
49:	learn: 3787.3416882	total: 92.6ms	remaining: 463ms
50:	learn: 3774.7620434	total: 95.4ms	remaining: 466ms
51:	learn: 3764.7214174	total: 97.7ms	remaining: 466ms
52:	learn: 3758.3416726	total: 99ms	remaining: 462ms
53:	learn: 3740.6465899	total: 101ms	remaining: 461ms
54:	learn: 3733.6149451	total: 103ms	remaining: 460ms
55:	learn: 3710.4028324	total: 106ms	remaining: 460ms
56:	learn: 3709.4468703	total: 106ms	remaining: 452ms
57:	learn: 3697.0335559	total: 108ms	remaining: 451ms
58:	learn: 3673.7810770	total: 110ms	remaining: 449ms
59:	learn: 3673.3309493	total: 110ms	remaining: 442ms
60:	learn: 3643.6707170	total: 113ms	remaining: 441ms
61:	learn: 3625.0605486	total: 115ms	remaining: 440ms
62:	learn: 3603.9406723

206:	learn: 2435.0835811	total: 432ms	remaining: 194ms
207:	learn: 2424.0638433	total: 435ms	remaining: 192ms
208:	learn: 2418.0374769	total: 437ms	remaining: 190ms
209:	learn: 2413.3235094	total: 439ms	remaining: 188ms
210:	learn: 2404.5691887	total: 441ms	remaining: 186ms
211:	learn: 2400.2057357	total: 443ms	remaining: 184ms
212:	learn: 2388.9113480	total: 446ms	remaining: 182ms
213:	learn: 2383.8396919	total: 449ms	remaining: 180ms
214:	learn: 2372.6136861	total: 451ms	remaining: 178ms
215:	learn: 2370.8440041	total: 454ms	remaining: 176ms
216:	learn: 2370.4779294	total: 456ms	remaining: 174ms
217:	learn: 2361.5362713	total: 458ms	remaining: 172ms
218:	learn: 2353.7797033	total: 461ms	remaining: 170ms
219:	learn: 2346.1811778	total: 463ms	remaining: 168ms
220:	learn: 2340.1029833	total: 466ms	remaining: 166ms
221:	learn: 2332.7722696	total: 468ms	remaining: 164ms
222:	learn: 2322.3102307	total: 470ms	remaining: 162ms
223:	learn: 2314.4011281	total: 472ms	remaining: 160ms
224:	learn

177:	learn: 4202.0304216	total: 95.5ms	remaining: 11.8ms
178:	learn: 4200.7774316	total: 96ms	remaining: 11.3ms
179:	learn: 4198.6894275	total: 96.6ms	remaining: 10.7ms
180:	learn: 4197.1638514	total: 97.2ms	remaining: 10.2ms
181:	learn: 4193.7490379	total: 97.8ms	remaining: 9.67ms
182:	learn: 4192.7063601	total: 98.3ms	remaining: 9.13ms
183:	learn: 4190.6736339	total: 98.7ms	remaining: 8.59ms
184:	learn: 4188.0763233	total: 99.2ms	remaining: 8.04ms
185:	learn: 4185.8205786	total: 99.7ms	remaining: 7.5ms
186:	learn: 4182.2450502	total: 100ms	remaining: 6.97ms
187:	learn: 4181.0736196	total: 101ms	remaining: 6.43ms
188:	learn: 4177.3549666	total: 101ms	remaining: 5.89ms
189:	learn: 4174.3069722	total: 102ms	remaining: 5.35ms
190:	learn: 4171.1903304	total: 102ms	remaining: 4.81ms
191:	learn: 4169.8610124	total: 103ms	remaining: 4.27ms
192:	learn: 4169.6978824	total: 103ms	remaining: 3.73ms
193:	learn: 4168.2600306	total: 103ms	remaining: 3.2ms
194:	learn: 4167.3933450	total: 104ms	remai

In [14]:

base_model = AdaBoostRegressor()
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1.0]
}

grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

best_model = grid_search.best_estimator_

print(f"Best Model -  R^2: {r2:.5f}, MSE: {mse:.5f}, MAE: {mae:.5f}")


Fitting 3 folds for each of 9 candidates, totalling 27 fits
Best Hyperparameters: {'learning_rate': 0.01, 'n_estimators': 50}
Best Model -  R^2: 0.87736, MSE: 17981892.50068, MAE: 2380.33282


In [15]:

param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(estimator=DecisionTreeRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=5,  
                           scoring='neg_mean_squared_error',  
                           n_jobs=-1) 

grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Hyperparameters: {best_params}")

print(f"Best Model -  R^2: {r2:.5f}, MSE: {mse:.5f}, MAE: {mae:.5f}")


Best Hyperparameters: {'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Model -  R^2: 0.84397, MSE: 22877589.83508, MAE: 2380.33282


C:\python3.10.1\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


In [16]:

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestRegressor()

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {best_params}")

print(f"Best Model -  R^2: {r2:.5f}, MSE: {mse:.5f}, MAE: {mae:.5f}")


Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
Best Model -  R^2: 0.87176, MSE: 18802520.31006, MAE: 2457.20834


In [17]:
regression_models = {
    "Linear Regression": LinearRegression(fit_intercept =  True),
    
    "Decision Tree Regressor": DecisionTreeRegressor(max_depth= 5, max_features= 'auto' , min_samples_leaf = 4, min_samples_split = 2),
    "Random Forest Regressor": RandomForestRegressor(max_depth = 20, min_samples_leaf = 4, min_samples_split = 10, n_estimators = 200),
    
    "CatBoost Regressor": CatBoostRegressor(depth = 4, iterations = 200, learning_rate = 0.05),
    "AdaBoost Regressor": AdaBoostRegressor(learning_rate = 0.01, n_estimators = 50)
}

for model_name, model in regression_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    

    print(f"Model: {model_name}")
    print(f"R^2 Score: {r2:.5f}")
    print(f"Mean Squared Error: {mse:.5f}")
    print(f"Mean Absolute Error: {mae:.5f}")
    
    print('***********************************')

Model: Linear Regression
R^2 Score: 0.76946
Mean Squared Error: 33802383.55494
Mean Absolute Error: 4154.70039
***********************************
Model: Decision Tree Regressor
R^2 Score: 0.85944
Mean Squared Error: 20609959.48593
Mean Absolute Error: 2647.56418
***********************************


C:\python3.10.1\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


Model: Random Forest Regressor
R^2 Score: 0.87072
Mean Squared Error: 18955161.43001
Mean Absolute Error: 2443.76480
***********************************
0:	learn: 11687.9523493	total: 1.15ms	remaining: 229ms
1:	learn: 11275.3168723	total: 1.75ms	remaining: 173ms
2:	learn: 10861.5137578	total: 2.43ms	remaining: 160ms
3:	learn: 10561.2099568	total: 2.84ms	remaining: 139ms
4:	learn: 10219.8247166	total: 3.53ms	remaining: 138ms
5:	learn: 9873.1602010	total: 4.06ms	remaining: 131ms
6:	learn: 9541.1118180	total: 4.86ms	remaining: 134ms
7:	learn: 9236.3983451	total: 5.46ms	remaining: 131ms
8:	learn: 8938.6022094	total: 6.01ms	remaining: 128ms
9:	learn: 8646.5379154	total: 6.46ms	remaining: 123ms
10:	learn: 8410.9411301	total: 6.97ms	remaining: 120ms
11:	learn: 8155.8356697	total: 7.51ms	remaining: 118ms
12:	learn: 7931.1213869	total: 8.01ms	remaining: 115ms
13:	learn: 7722.2904490	total: 8.52ms	remaining: 113ms
14:	learn: 7503.1725546	total: 9.21ms	remaining: 114ms
15:	learn: 7318.8478102	tot

Model: AdaBoost Regressor
R^2 Score: 0.84603
Mean Squared Error: 22576071.68412
Mean Absolute Error: 3102.86989
***********************************


In [18]:
import pandas as pd

results = []

for model_name, model in regression_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    results.append({
        "Model": model_name,
        "R^2 Score": r2,
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae
    })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Sort the DataFrame by R^2 score in descending order
results_df_sorted = results_df.sort_values(by='R^2 Score', ascending=False)

# Print the sorted DataFrame
print(results_df_sorted)


C:\python3.10.1\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


0:	learn: 11687.9523493	total: 992us	remaining: 197ms
1:	learn: 11275.3168723	total: 1.67ms	remaining: 165ms
2:	learn: 10861.5137578	total: 2.33ms	remaining: 153ms
3:	learn: 10561.2099568	total: 2.73ms	remaining: 134ms
4:	learn: 10219.8247166	total: 3.24ms	remaining: 127ms
5:	learn: 9873.1602010	total: 3.71ms	remaining: 120ms
6:	learn: 9541.1118180	total: 4.23ms	remaining: 117ms
7:	learn: 9236.3983451	total: 4.68ms	remaining: 112ms
8:	learn: 8938.6022094	total: 5.16ms	remaining: 109ms
9:	learn: 8646.5379154	total: 5.62ms	remaining: 107ms
10:	learn: 8410.9411301	total: 6.11ms	remaining: 105ms
11:	learn: 8155.8356697	total: 6.65ms	remaining: 104ms
12:	learn: 7931.1213869	total: 7.08ms	remaining: 102ms
13:	learn: 7722.2904490	total: 7.69ms	remaining: 102ms
14:	learn: 7503.1725546	total: 8.22ms	remaining: 101ms
15:	learn: 7318.8478102	total: 8.64ms	remaining: 99.4ms
16:	learn: 7121.0031518	total: 9.12ms	remaining: 98.2ms
17:	learn: 6940.5422432	total: 9.61ms	remaining: 97.1ms
18:	learn: 67

                     Model  R^2 Score  Mean Squared Error  Mean Absolute Error
3       CatBoost Regressor   0.877361        1.798189e+07          2380.332821
2  Random Forest Regressor   0.870880        1.893207e+07          2439.209665
1  Decision Tree Regressor   0.859437        2.060996e+07          2647.564182
4       AdaBoost Regressor   0.848077        2.227568e+07          3108.565967
0        Linear Regression   0.769463        3.380238e+07          4154.700387


# catboost regressor is best model after doing hyperparameter tunning based on r2 score

In [19]:
import pickle

model = CatBoostRegressor(depth = 4, iterations = 200, learning_rate = 0.05)
model.fit(X_train, y_train)

# Save the model to a file using pickle
with open('catboost_model.pkl', 'wb') as file:
    pickle.dump(model, file)

# Load the model from the file
with open('catboost_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)




0:	learn: 11687.9523493	total: 963us	remaining: 192ms
1:	learn: 11275.3168723	total: 1.6ms	remaining: 158ms
2:	learn: 10861.5137578	total: 2.17ms	remaining: 143ms
3:	learn: 10561.2099568	total: 2.67ms	remaining: 131ms
4:	learn: 10219.8247166	total: 3.19ms	remaining: 124ms
5:	learn: 9873.1602010	total: 3.69ms	remaining: 119ms
6:	learn: 9541.1118180	total: 4.16ms	remaining: 115ms
7:	learn: 9236.3983451	total: 4.63ms	remaining: 111ms
8:	learn: 8938.6022094	total: 5.08ms	remaining: 108ms
9:	learn: 8646.5379154	total: 5.56ms	remaining: 106ms
10:	learn: 8410.9411301	total: 6.05ms	remaining: 104ms
11:	learn: 8155.8356697	total: 6.5ms	remaining: 102ms
12:	learn: 7931.1213869	total: 6.97ms	remaining: 100ms
13:	learn: 7722.2904490	total: 7.52ms	remaining: 99.9ms
14:	learn: 7503.1725546	total: 8.26ms	remaining: 102ms
15:	learn: 7318.8478102	total: 8.67ms	remaining: 99.7ms
16:	learn: 7121.0031518	total: 9.21ms	remaining: 99.1ms
17:	learn: 6940.5422432	total: 9.74ms	remaining: 98.5ms
18:	learn: 676